In [1]:
import pandas as pd
import numpy as np
import get_prices as hist
import tensorflow as tf
from preprocessing import DataProcessing
import pandas_datareader.data as pdr
import yfinance as fix
import matplotlib.pyplot as plt
import WXBizSendMsg

In [2]:
fix.pdr_override()

start = "2000-01-01"
end = "2019-08-22"

stock="000001.SS"

hist.get_stock_data(stock, start_date=start, end_date=end)
process = DataProcessing("stock_prices.csv", 0.9)

process.gen_test(10)
process.gen_train(10)


X_train = process.X_train / np.array([process.value_max, process.volume_max])  # 归一化， 包括Adj Close 和 Volume
Y_train = process.Y_train / process.value_max

X_test = process.X_test / np.array([process.value_max, process.volume_max])
Y_test = process.Y_test / process.value_max


[*********************100%***********************]  1 of 1 downloaded


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(20, input_shape=(10, 2), return_sequences=True))
model.add(tf.keras.layers.LSTM(20))
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer="adam",loss="mean_squared_error")

h=model.fit(X_train, Y_train, epochs=50,shuffle=True,batch_size=1)

W0823 16:18:53.999618 140094662838080 deprecation.py:506] From /opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0823 16:18:55.114606 140094662838080 deprecation.py:323] From /opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
4379/4379 [==============================] - 118s 27ms/sample - loss: 8.8973e-04
Epoch 2/50
4379/4379 [==============================] - 110s 25ms/sample - loss: 3.9518e-04
Epoch 3/50
4379/4379 [==============================] - 109s 25ms/sample - loss: 2.9903e-04
Epoch 4/50
4379/4379 [==============================] - 101s 23ms/sample - loss: 2.4052e-04
Epoch 5/50
4379/4379 [==============================] - 104s 24ms/sample - loss: 2.2156e-04
Epoch 6/50
4379/4379 [==============================] - 97s 22ms/sample - loss: 2.1606e-04
Epoch 7/50
4379/4379 [==============================] - 99s 23ms/sample - loss: 1.9914e-04
Epoch 8/50
3046/4379 [===================>..........] - ETA: 29s - loss: 2.1738e-04

In [ ]:
print("history.loss:",h.history)
plt.title('loss')
losses=h.history['loss']
plt.plot(losses, label='loss')
plt.legend()
plt.show()

print(model.evaluate(X_test, Y_test))

X_predict = model.predict(X_test)
plt.title(start+"—"+end)
plt.plot(Y_test * process.value_max, label="Actual", c="blue")
plt.plot(X_predict * process.value_max, label="Predict", c="red")
plt.legend()
plt.savefig('stock.png')
plt.show()

In [ ]:
data = pdr.get_data_yahoo(stock, "2019-08-09", "2019-08-22")
stock = data[["Adj Close", "Volume"]]
X1_predict = np.array(stock) / np.array([process.value_max, process.volume_max])
X1_predict = X1_predict.reshape(1, -1, 2)

print("predict:")
predict=model.predict(X1_predict) * process.value_max
print(predict)

In [ ]:
api = WXBizSendMsg.WXBizSendMsg(WXBizSendMsg.sCorpID,WXBizSendMsg.corpsecret)
#发送图片
media_id = api.upload_media("file","./stock.png")
message = WXBizSendMsg.ImageMessage(WXBizSendMsg.agentID,touser="@all",media_id=media_id)
res = api.send_message(message)
print(res)#

predict="{:.2f}".format(predict[0][0])
message = WXBizSendMsg.TextMessage(WXBizSendMsg.agentID,touser='@all',content="明日上证指数:"+predict)
res = api.send_message(message)
print(res)